## Causal mask
---

causal mask มีไว้เพื่อบังคับ auto regression ที่ position t ให้มองเห็นได้แค่ $\leq t$ ไม่ให้เห็น $t + 1, t + 2, ...$

สมมุติ ตัวอย่าง

> * T = 4 จำนวน token มากสุด 4 ตัว
> * row = query position
> * col = key position

กฏคือ $ \text{col} \leq \text{row}$

> * q0: $\begin{bmatrix}1& 0& 0& 0\end{bmatrix}$  
> * q1: $\begin{bmatrix}1& 1& 0& 0\end{bmatrix}$  
> * q2: $\begin{bmatrix}1& 1& 1& 0\end{bmatrix}$  
> * q3: $\begin{bmatrix}1& 1& 1& 1\end{bmatrix}$  

ตำแหน่งที่เป็น 0 จะถูกบวกด้วย $-\infty$ ก่อนการทำ softmax

### ตำแหน่งที่ใช้ casual mask

* ใน transformer block ที่ self-attention เดิมเรามี 

$$ scores = \frac {Q \cdot K^T} {\sqrt{d_k}}$$

จากนั้นเรานำ socore ที่ได้

$$ scores = scores + causal\_mask$$

เมื่อ
* $Q = W_Q \cdot X$ &rarr; Q คือ query 
* $K = W_K \cdot X$ &rarr; K คือ key 
* $d_k = \frac {\text{parameter number of model}} {\text{number of attention head}}$

### code example 

In [3]:
import numpy as np

In [10]:
def make_causal_mask(T, neg_large = -1e9):
    # upper triangle (k > T ) = block
    m = np.triu(np.ones((T, T), dtype=np.float64), k = 1)
    mask = m * neg_large
    return mask # (T, T)



In [11]:
def softmax(x, axis = -1) :
    x = x - np.max(x, axis=axis, keepdims=True);
    ex = np.exp(x)
    return ex / np.sum(ex, axis=axis, keepdims=True)

In [12]:
scores = np.random.rand(4, 4)
print(scores)

[[0.125646   0.96397371 0.33659327 0.43928826]
 [0.13430261 0.97102916 0.70184706 0.45541943]
 [0.74333667 0.54980993 0.01403779 0.57009106]
 [0.17762828 0.41669086 0.89345958 0.94088805]]


In [ ]:
T = scores.shape[0]
causal_mask = make_causal_mask(T)
masked_scores = scores + causal_masks
A = softmax(masked_scores)

In [14]:
print("causal_mask=\n", causal_mask)
print("A=\n", A)

causal_mask=
 [[-0.e+00 -1.e+09 -1.e+09 -1.e+09]
 [-0.e+00 -0.e+00 -1.e+09 -1.e+09]
 [-0.e+00 -0.e+00 -0.e+00 -1.e+09]
 [-0.e+00 -0.e+00 -0.e+00 -0.e+00]]
A=
 [[1.         0.         0.         0.        ]
 [0.30222466 0.69777534 0.         0.        ]
 [0.43359592 0.35730376 0.20910032 0.        ]
 [0.15476995 0.19656681 0.31664178 0.33202145]]
